# WORKER | Agentic Pattern

**Worker** is a single-agent pattern in which a task is assigned to a capable agent (the Worker), who interprets the task and completes it to the best of its ability. The result is returned directly without any internal validation loop or coordination with other agents.

## Problem Addressed
This pattern is ideal for tasks that are self-contained, well-scoped, and do not require iterative refinement, delegation, or oversight. It simplifies agent orchestration by relying on a single, well-configured agent to interpret and execute the task autonomously.

## Pattern Structure

**Agents |**
- `Worker`: Responsible for interpreting and executing the task, and returning the result.

**Coordination Topology |**
- Single-agent execution. Input is passed to the Worker; output is returned to the caller. No feedback or retries are built into the pattern.

## Assumptions

- The Worker agent:
  - Possesses the appropriate **tools** or **capabilities** to complete the task.
  - Is capable of **interpreting the input**, **executing actions**, and **generating a coherent result**.
  - May or may not use memory (STM or LTM), depending on task requirements.
- There is no expectation of **external validation**, **collaboration**, or **multi-step refinement**.

## Inputs
- A task description, instruction, or goal (typically natural language or structured prompt).
- Optional: Memory, which gives the Worker the ability to retain a  history of past work as context for future tasks.

## Outputs
- A final result, as produced by the Worker.
- May optionally include a self-assessed confidence score, rationale, or explanation.

## Behavioural Flow

1. Input task is provided to the Worker.
2. Worker interprets the task and determines an appropriate course of action.
3. Worker executes the task using internal reasoning and available tools.
4. Worker returns the result.

## Strengths
- Simple to implement and operationalize.
- Low latency and cost.
- Suitable for tasks with low ambiguity or limited risk from error.

## Weaknesses
- No built-in feedback, validation, or improvement loop.
- Prone to hallucination or misinterpretation on complex or ambiguous inputs.
- No accountability or explanation beyond what the Worker may optionally provide.

## Variations and Extensions
- **Reflective Worker**: Adds an internal self-critique step before returning.
- **Memory-Augmented Worker**: Leverages LTM for more consistent outputs across tasks.

## Example Use Cases
- Generate a product description from specifications.
- Answer a factual question using embedded knowledge.
- Write a function to perform a specific operation.

---

## Implementation

Create a `.env` file in the root directory of this project for private configuration variables (API keys, etc). Running this notebook requires a valid `OPENAI_API_KEY=sk-proj-XXXX`.

In [ ]:
# Load the environment variables
from dotenv import load_dotenv

load_dotenv(override=True)

The worker agent can be configured with a name, instructions, model and other settings, as shown in the following cell. We construct the session name by appending a unique `datetime`-based string to the worker's name.

In [ ]:
# Configuration for the Worker agent

from agents import ModelSettings
from datetime import datetime

agent_name = "Basic Worker"

agent_instructions = """
You are a single-step Worker. Execute the given task directly and return the best possible result.
Do not ask clarifying questions. Do not handoff, delegate, or call tools unless explicitly provided.
Prefer concise, correct outputs. Use Markdown formatting when helpful.
"""

agent_model = "gpt-4o-mini"

agent_model_settings=ModelSettings(
    model=agent_model,
    temperature=0.2,
    max_tokens=1000
)

agent_session_name  = agent_name + datetime.now().strftime("%Y-%m-%dT%H:%M:%SU%s")

Next, we instantiate the worker as an OpenAI Agent from the `openai-agents` package.

In [ ]:
# Define the Worker agent

from agents import Agent, Runner, trace

worker = Agent(
    name=agent_name,
    instructions=agent_instructions,
    model_settings=agent_model_settings
)

A worker agent can be given whatever capabilities it needs to get the job done, including memory and tools. For this example, we give it a Session that is mainatined as an in-memory SQLite database.

In [ ]:
# Give the agent the ability to remember from one interaction to the next.

from agents import SQLiteSession

agent_session = SQLiteSession(agent_session_name)

This is the method that is used to assign a task to the worker and return the results. Note that it uses the asyncio coroutine framework, and is marked `async`; therefore assign the return value using `await`.

In [ ]:
async def assign_task(task: str, enable_memory: bool) -> str:
    """
    Asssign a task to the agent and receive its response in return.
    """

    # Check that Worker agent exists
    if worker is None:
        return "Agent has not been created yet."
        
    try:
        result = None
        with trace(worker.name):
            if enable_memory:
                result = await Runner.run(worker, task, session=agent_session)
            else:
                result = await Runner.run(worker, task)
        return result.final_output
    except Exception as e:
        return f"Error: {e}"


The methods above can be tested with the gradio interface below; alternatively, you can use the `WorkerPattern` class instead. The class combines all of the above code snippets into a nicely defined unit.

In [ ]:
from agents import Agent, ModelSettings, Runner, SQLiteSession, trace
from datetime import datetime


class WorkerPattern:

    def __init__(self, agent_name: str = None, agent_instructions: str = None, 
                 agent_model: str = "gpt-4o-mini", 
                 agent_temperature: float = 0.2, 
                 agent_max_tokens: int = 1000,
                 agent_has_memory: bool = False) -> None:
        """
        Initialize the object and set the private parameters that govern the worker agent.
        """

        # Generate a timestamp string for unique naming
        now_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%SU%s")

        # Assign the agent a name
        self.__agent_name = agent_name or f"WorkerAgent_{now_string}"
        
        # Generate a unique memory Session name
        self.__agent_session_name = f"{self.__agent_name}__SESSION_{now_string}"

        # Set the agent's instructions prompt
        self.__agent_instructions = agent_instructions or """
            You are a single-step Worker. Execute the given task directly and return the best possible result.
            Do not ask clarifying questions. Do not handoff, delegate, or call tools unless explicitly provided.
            Prefer concise, correct outputs. Use Markdown formatting when helpful.
        """

        # Set additional agent parameters
        self.__agent_model = agent_model
        self.__agent_temperature = float(agent_temperature)
        self.__agent_max_tokens = int(agent_max_tokens)
        self.__agent_has_memory = bool(agent_has_memory)

        self.__worker_agent = None
        self.__agent_session = None


    def __create_agent(self):
        """
        Create the worker agent and a session (if the agent is meant to have memory).
        """

        agent_model_settings = ModelSettings(
            temperature=self.__agent_temperature,
            max_tokens=self.__agent_max_tokens
        )

        self.__worker_agent = Agent(
            name=self.__agent_name,
            instructions=self.__agent_instructions,
            model=self.__agent_model,
            model_settings=agent_model_settings
        )

        if self.__agent_has_memory:
            self.__agent_session = SQLiteSession(self.__agent_session_name)
        

        
    async def assign_task(self, task: str, enable_memory: bool) -> str:
        """
        Asssign a task to the agent and receive its response in return.
        """

        # Check that Worker agent exists
        if self.__worker_agent is None:
            self.__create_agent()
            
        try:
            result = None
            with trace(self.__worker_agent.name):
                if enable_memory:
                    result = await Runner.run(self.__worker_agent, task, session=self.__agent_session)
                else:
                    result = await Runner.run(self.__worker_agent, task, session=None)
            return result.final_output
        except Exception as e:
            return f"Error: {e}"
    

The gardio code below produces a handy user interface for testing either the standalone cells above or the `WorkerPattern` class. Set the `use_worker_pattern` to `True` to test the class; `False` will test the code in the separate cells. In all cases, be sure to run the cell that loads the `.env` file.

In [ ]:
# Create a gradio interface to interact with the worker agent

import gradio as gr

# Set this to True if you want to test WorkerPattern object, False to test the standalone methods.
use_worker_pattern = False

with gr.Blocks(title="Worker — Agentic Design Patterns") as demo:

    wp = WorkerPattern(agent_has_memory=True) if use_worker_pattern else None

    gr.Markdown("""
    # 🛠️ Worker Pattern
    
    This is a simple worker agent that can be used to perform single-step execution tasks without delegation or validation. Ask, and it will do its best to complete the task.
    """)

    with gr.Column():
        with gr.Row():
            task = gr.Textbox(label="Task", placeholder="What do you want the Worker to do?", lines=4, scale=9)
            with gr.Column(scale=1):
                enable_memory = gr.Checkbox(label="Remember Stuff")
                run_btn = gr.Button("Ask", variant="primary", scale=1)
        out = gr.Markdown(label="Result")

    async def on_run(task:str, memory: bool):
        run_btn.interactive = False  # disable button while running
        try:
            if wp:
                return await wp.assign_task(task, memory)
            else:
                return await assign_task(task, memory)
        finally:
            run_btn.interactive = True  # re-enable button

    run_btn.click(on_run, inputs=[task, enable_memory], outputs=[out])
    task.submit(on_run, inputs=[task, enable_memory], outputs=[out])

demo.launch(inline=True)

**Copyright (c) 2025 Aluance.** All Rights Reserved.